<a href="https://colab.research.google.com/github/andreusjh99/Learning-Tensorflow2.0/blob/master/Saving_and_loading_models_(Keras_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Saving and loading models & checkpoints (Basics)**
Author: [Chia Jing Heng](https://github.com/andreusjh99)

This notebook explores the basics of using the keras API to save and load models. The bulk of this tutorial is from the tensorflow official guides [here](https://www.tensorflow.org/guide/keras/save_and_serialize) and [here](https://www.tensorflow.org/tutorials/keras/save_and_load).

Model progress can be saved before, during and/or training. This means a model can be reused, can resume where it left off or can be used to make predictions after trained.

A Keras model consists of multiple components:
* An **architecture/configuration**, which specifies the layers and how they are connected.
* A set of **weights** (the *state of the model*)
* An **optimiser** and its state
* A set of **losses** and **metrics**.

The Keras API makes it possible to save these to disk. Most commonly, people save the entire model or only the weights. In this notebook, we will explore both.

#### Setup

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.2.0


### Mounting Google Drive
This step is important for Google Colab. This enables you to save your models to Google drive.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
path = "/content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/Models/" 

#### Creating the model


In [6]:
# Using the Sequential API
def create_model():
    model = keras.Sequential(name = "Classifier")
    model.add(keras.Input(shape = (784,), name = "Digits"))
    model.add(layers.Dense(64, activation = "relu", name = "Input_layer"))
    model.add(layers.Dense(64, activation = "relu", name = "Hidden_dense_1"))
    model.add(layers.Dense(10, activation = "softmax", name = "predictions"))

    return model

# Save and load model

The model could be saved at any stage of your project. You can save and load a model before compiling, after compiling but before fitting, during fitting, and/or after fitting.

*Note: usually you would save a model before compiling only if you want to use the architecture but don't want to use the same optimiser(s)/loss(es)/metric(s).

There are 2 formats you can use to save an entire model to disk:
1. tf SavedModel format (default and recommended)
2. keras H5 format (old)

The default SavedModel is recommended as the older keras H5 format doesn't save custom objects like custom layers.

**SavedModel**

The SavedModel format is a folder containing:
* `assets` folder
* `saved_model.pb`
* `variables` folder

The model architecture, and training configuration (including optimiser, losses, metrics etc) are stored in `saved_model.pb` while the weights are saved in the `variables` directory.

## `model.save()` and `load_model()`

Apart from during fitting, the model could be saved by using a simple command:

        model.save(model_dir)

The model can then be loaded with:

        tf.keras.models.load_model(model_dir)

In [7]:
model1 = create_model()
model1.summary()

Model: "Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (Dense)          (None, 64)                50240     
_________________________________________________________________
Hidden_dense_1 (Dense)       (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
# save before compiling
model1.save(path + "model1_before_compiled")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/Models/model1_before_compiled/assets


In [11]:
# load model
model2 = tf.keras.models.load_model(path + "model1_before_compiled")
model2.summary()

Model: "Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (Dense)          (None, 64)                50240     
_________________________________________________________________
Hidden_dense_1 (Dense)       (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


To save or load model after compiling, fitting etc., just use the same commands.

## `keras.callbacks.ModelCheckpoint()`

To save the model during training, you need to use callbacks. This is also called saving checkpoints of your model.

The following two cells are just setting up.

In [8]:
model1.compile(
    optimizer="rmsprop", 
    loss="sparse_categorical_crossentropy", 
    metrics=["sparse_categorical_accuracy"]
)

In [9]:
# load the Mnist data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255 # reshape, cast and normalise
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32") # cast
y_test = y_test.astype("float32")

11493376/11490434 [==============================] - 0s 0us/step


The 2 cells are setup stuff. Now to the real business.

Define your callbacks before fitting your model. For more info about callbacks, refer to this tutorial.

In [12]:
# define callbacks list
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = path + "model1_{epoch}",
        monitor = "val_loss",
        save_best_only = True,
        verbose = 1,
    )
]

`keras.callbacks.ModelCheckpoint()` takes in:
* `filepath`
* `monitor`: quantity to monitor 
* `save_best_only`: if True, only save model if the model improved (judged by the monitored quantity). False by default.
* `save_weights_only`: if True, only save the weights. False by default.
* `mode`: `"max"` means maximisation of monitored quantity; `"min"` means minimisation; `"auto"` by default, i.e. keras figures it out from the name of the quantity.
* `save_freq`: `"epoch"` means per epoch; integer means number of batches.

So to save the entire model, `save_weights_only` should be False.



In [14]:
num_epochs = 6
batch_size = 64

train_log = model1.fit(
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = num_epochs,
    validation_split = 0.2,
    callbacks = callbacks
)

Epoch 1/6
725/750 [============================>.] - ETA: 0s - loss: 0.3488 - sparse_categorical_accuracy: 0.9022
Epoch 00001: val_loss improved from inf to 0.18568, saving model to /content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/Models/model1_1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/Models/model1_1/assets
750/750 [==============================] - 3s 4ms/step - loss: 0.3446 - sparse_categorical_accuracy: 0.9033 - val_loss: 0.1857 - val_sparse_categorical_accuracy: 0.9471
Epoch 2/6
732/750 [============================>.] - ETA: 0s - loss: 0.1638 - sparse_categorical_accuracy: 0.9507
Epoch 00002: val_loss improved from 0.18568 to 0.13333, saving model to /content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/Models/model1_2
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Colab Notebooks/Tensorflow2.0_Tutorial/M

# Save and load weights

You could choose to save only the weights and you could do this at any stage as well.

There are two formats you can save weights to disk:
1. Tensorflow Checkpoint (Default, recommended)
2. HDF5 (Old)

The Tensorflow Checkpoint dir contains:
* one or more shards that contain your model's weights.
* An index file that indicates which weights are stored in which shard.

This folder is the same as the `Variables` folder in the SavedModel folder created as you saved the entire model.

## `model.save_weights()` and `model.load_weights()`

Save weights by:

        model.save_weights(filepath)

Load weights by:

        model.load_weights(filepath)

An alternative to loading weights: use `model.get_weights()` with `model.load_weights()`. `get_weights()` is a method to get the weights from the model. It loads the weights from the `variable` dir in your SavedModel dir. So to do this:

        model_1 = tf.keras.models.load_model(model_dir)
        model_2.load_weights(model_1.get_weights())

This means you could essentially load weights from a saved model or load weights from saved weights of a model. Note, the architectures of the models need to be compatible. 




In [15]:
model3 = create_model()
model3.summary()

Model: "Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (Dense)          (None, 64)                50240     
_________________________________________________________________
Hidden_dense_1 (Dense)       (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
dir = path + "model3_weights/"
model3.save_weights(dir + "model3_before_compiled")

In [18]:
model3.compile(
    optimizer="rmsprop", 
    loss="sparse_categorical_crossentropy", 
    metrics=["sparse_categorical_accuracy"]
)

model3.load_weights(dir + "model3_before_compiled")

## `keras.callbacks.ModelCheckpoint()`

To save the weights during training, you need to use callbacks. This is also called saving checkpoints of your model.

This is similar to saving the entire model, just setting `save_weights_only` to `True`.

In [19]:
# define callbacks list
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = dir + "checkpoints/model3_epoch{epoch}",
        monitor = "val_loss",
        save_best_only = True,
        save_weights_only = True,
        verbose = 1,
    )
]

In [21]:
num_epochs = 6
batch_size = 64

train_log = model3.fit(
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = num_epochs,
    validation_split = 0.2,
    callbacks = callbacks
)

Epoch 1/6
729/750 [============================>.] - ETA: 0s - loss: 0.3532 - sparse_categorical_accuracy: 0.8982
Epoch 00001: val_loss did not improve from 0.10518
750/750 [==============================] - 2s 3ms/step - loss: 0.3489 - sparse_categorical_accuracy: 0.8993 - val_loss: 0.1847 - val_sparse_categorical_accuracy: 0.9461
Epoch 2/6
743/750 [============================>.] - ETA: 0s - loss: 0.1646 - sparse_categorical_accuracy: 0.9509
Epoch 00002: val_loss did not improve from 0.10518
750/750 [==============================] - 2s 2ms/step - loss: 0.1644 - sparse_categorical_accuracy: 0.9510 - val_loss: 0.1429 - val_sparse_categorical_accuracy: 0.9584
Epoch 3/6
743/750 [============================>.] - ETA: 0s - loss: 0.1200 - sparse_categorical_accuracy: 0.9642
Epoch 00003: val_loss did not improve from 0.10518
750/750 [==============================] - 2s 2ms/step - loss: 0.1196 - sparse_categorical_accuracy: 0.9643 - val_loss: 0.1262 - val_sparse_categorical_accuracy: 0.964